In [1]:
#Import relevant modules
import time
import re
import pandas as pd
import requests
import datetime
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By

In [2]:
# get coming race day from schedule csv
df = pd.read_csv('../data/schedule/schedule_s2021.csv')
df.columns = ['year','month','day']
df['next_race_day']=pd.to_datetime(df)
min_idx=df[df['next_race_day']>=pd.Timestamp('today').floor('D')]['next_race_day'].idxmin()
next_race_day=df.loc[min_idx,'next_race_day']
coming_race_day=str(next_race_day.year)+'-'+str(next_race_day.month)+'-'+str(next_race_day.day)
coming_race_day

'2021-1-10'

In [3]:
driver = webdriver.Chrome('/Users/Hei/Applications/chromedriver')

In [4]:
coming_race_day
suffix_raceno='1' #hardcode for test, default =1
URL_coming_race = 'https://bet.hkjc.com/racing/pages/odds_wp.aspx?lang=en&date='+coming_race_day+'&raceno='+suffix_raceno
URL_coming_race

'https://bet.hkjc.com/racing/pages/odds_wp.aspx?lang=en&date=2021-1-10&raceno=1'

In [5]:
# Initialize soup for getting number of race
driver.get(URL_coming_race)
time.sleep(5)
subhtml = driver.page_source
soup = BeautifulSoup(subhtml, 'html.parser')

# Initialize dataframe
df_race = pd.DataFrame(columns=['season','race','horse_no','horse','jockey','trainer','class','track','location',\
                                'actual_weight','draw','distance_m','win_odds','date'])

# Get location
location=soup.find('div',{'class':'mtgInfoDV'}).text.split(',')[2].strip()

In [6]:
def get_no_of_race():
    racebg = soup.find(class_="racebg").contents
    count_race=0
    for i in range(len(racebg)):
        if racebg[i] !='\n' and len(racebg[i].attrs)==4 and 'id' in racebg[i].attrs.keys():
            count_race+=1
    return count_race

def get_race_info():
    if location=='Sha Tin':
        pattern=[p.text for p in soup.find_all('strong')]
        line=' '.join(pattern)
        match_str = re.search(r'Race\s(\d+)', line)
        race=match_str.group(1)
        race_info=soup.find('span',{'class':'content'}).text.split(',')
        date=race_info[1].strip()
        race_class=race_info[3].strip()
        if len(race_info)==7:
            track=race_info[4].strip()
        else:
            track=race_info[4].strip()+' '+race_info[5].strip()
        distance=race_info[-2].strip()[:-1]
    else:
        pattern=[p.text for p in soup.find_all('strong')]
        line=' '.join(pattern)
        match_str = re.search(r'Race\s(\d+)', line)
        race=match_str.group(1)
        race_info=soup.find('span',{'class':'content'}).text.split(',')
        date=race_info[1].strip()
        race_class=race_info[3].strip()
        turf=race_info[4].strip()
        course=race_info[5].strip()
        track=turf+' - '+course
        distance=race_info[6].strip()[:-1]
    return race,date,race_class,track,distance

def write_table():
    race_no,date,race_class,track,distance=get_race_info()
    dict_table={'season':[],'race':[],'horse_no':[],'horse':[],'jockey':[],'trainer':[],'class':[],'track':[],\
                'location':[],'actual_weight':[],'draw':[],'distance_m':[],'win_odds':[],'date':[]}
    table = soup.find_all(id='horseTable')[0].tbody.find_all('td')[:-6]
    for i in range(len(table)):
        if i %10==0:
            dict_table['season'].append('2020/2021')
            dict_table['race'].append(race_no)
            dict_table['class'].append(race_class)
            dict_table['track'].append(track)
            dict_table['location'].append(location)
            dict_table['distance_m'].append(distance)
            dict_table['date'].append(date)
            dict_table['horse_no'].append(table[i].text)
        elif i%10==2:
            dict_table['horse'].append(table[i].text)
        elif i%10==3:
            dict_table['draw'].append(table[i].text)
        elif i%10==4:
            dict_table['actual_weight'].append(table[i].text)
        elif i%10==5:
            dict_table['jockey'].append(table[i].text)
        elif i%10==6:
            dict_table['trainer'].append(table[i].text)
        elif i%10==7:
            dict_table['win_odds'].append(table[i].text)
    df=pd.DataFrame(dict_table)
    return df


In [7]:
for i in range(get_no_of_race()):
    # get soup for each race
    URL_coming_race = 'https://bet.hkjc.com/racing/pages/odds_wp.aspx?lang=en&date='+coming_race_day+'&raceno='+str(i+1)
    
    driver.get(URL_coming_race)
    time.sleep(5)
    subhtml = driver.page_source
    soup = BeautifulSoup(subhtml, 'html.parser')
    
    #get horse inforamtion for each race
    table = soup.find_all(id='horseTable')[0].tbody.find_all('td')[:-6]
    df_race=pd.concat([df_race,write_table()])


In [8]:
df_race['date'] = pd.to_datetime(df_race['date'], format='%d/%m/%Y')
df_race

,season,race,horse_no,horse,jockey,trainer,class,track,location,actual_weight,draw,distance_m,win_odds,date
0,2020/2021,1,1,BRANDING POWER,C Y Ho,K W Lui,Class 4,"TURF ""C"" Course",Sha Tin,129,5,1200,4.0,2021-01-10
1,2020/2021,1,2,ZONE D,J Moreira,J Size,Class 4,"TURF ""C"" Course",Sha Tin,129,12,1200,12,2021-01-10
2,2020/2021,1,3,FOREVER CITY,V Borges,P O'Sullivan,Class 4,"TURF ""C"" Course",Sha Tin,129,11,1200,12,2021-01-10
3,2020/2021,1,4,LUCKY FUN,Z Purton,C S Shum,Class 4,"TURF ""C"" Course",Sha Tin,129,4,1200,2.8,2021-01-10
4,2020/2021,1,5,MISTER ARM,H T Mo,Y S Tsui,Class 4,"TURF ""C"" Course",Sha Tin,129,9,1200,23,2021-01-10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9,2020/2021,10,10,WINNING METHOD,M Chadwick,C S Shum,Class 2,"TURF ""C"" Course",Sha Tin,120,13,1400,26,2021-01-10
10,2020/2021,10,11,LUCKY PATCH,M F Poon,K W Lui,Class 2,"TURF ""C"" Course",Sha Tin,117,3,1400,5.4,2021-01-10
11,2020/2021,10,12,BEAUTY JOY,A Badel,D A Hayes,Class 2,"TURF ""C"" Course",Sha Tin,116,5,1400,11,2021-01-10
12,2020/2021,10,13,BEAUTY SMILE,C Y Ho,A S Cruz,Class 2,"TURF ""C"" Course",Sha Tin,116,10,1400,13,2021-01-10


In [9]:
df_race.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 132 entries, 0 to 13
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   season         132 non-null    object        
 1   race           132 non-null    object        
 2   horse_no       132 non-null    object        
 3   horse          132 non-null    object        
 4   jockey         132 non-null    object        
 5   trainer        132 non-null    object        
 6   class          132 non-null    object        
 7   track          132 non-null    object        
 8   location       132 non-null    object        
 9   actual_weight  132 non-null    object        
 10  draw           132 non-null    object        
 11  distance_m     132 non-null    object        
 12  win_odds       132 non-null    object        
 13  date           132 non-null    datetime64[ns]
dtypes: datetime64[ns](1), object(13)
memory usage: 15.5+ KB


In [10]:
back_up='../data/latest/coming_race_'+str(next_race_day.year)+str(next_race_day.month)+str(next_race_day.day)+'.csv'
df_race.to_csv('../data/latest/coming_race.csv',index=False)
df_race.to_csv(back_up,index=False)